In [1]:
import math
from pymongo import MongoClient
from xml.etree import ElementTree
import pprint
import json 
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
import numpy as np
import urllib.request, json
import datetime, time
import operator
import pandas as pd
import numpy as np

In [2]:
start_time = time.time()
conn = MongoClient('120.126.136.17')
db = conn.Tracker
collection = [db.james]# db.db2, db.dn2, db.james, db.leo
userInfo = []
end = datetime.datetime.now().timestamp()
start = (datetime.datetime.now() + datetime.timedelta(days = -14)).timestamp()
for col in collection:
    cursor = col.find({'timestamp': {'$gte': start, '$lt': end}})
    df = pd.DataFrame(list(cursor))
df.replace('', np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)
df['latitude'] = df['latitude'].astype(float).round(4)
df['longitude'] = df['longitude'].astype(float).round(4)
print("--- %s seconds ---" % (time.time() - start_time))

--- 19.913961172103882 seconds ---


In [5]:
df.head()

,_id,day,hour,hr_value,latitude,longitude,minute,month,o2_value,second,step_value,timestamp,year
0,5cc25b80c4e5a668b7eff09a,26,9,54,24.9488,121.3741,14,4,97,16,17,1.556241e+09,2019
1,5cc25b8ac4e5a668b7eff09e,26,9,54,24.9488,121.3741,14,4,97,26,15,1.556241e+09,2019
2,5cc25b94c4e5a668b7eff0a2,26,9,54,24.9488,121.3741,14,4,97,36,14,1.556241e+09,2019
3,5cc25b9ec4e5a668b7eff0a6,26,9,54,24.9488,121.3741,14,4,97,46,16,1.556241e+09,2019
4,5cc25ba8c4e5a668b7eff0aa,26,9,69,24.9488,121.3741,14,4,95,56,12,1.556241e+09,2019


In [6]:
lattestData = df[-1:]
baseLocation = [lattestData['latitude'], lattestData['longitude']]
downlat = float(baseLocation[0]) - 0.01#24.938590 
leftlong = float(baseLocation[1]) - 0.01#121.360761

In [7]:
squarefreq = df.groupby(df[['longitude','latitude']].columns.tolist(),as_index=False).size()

In [62]:
squarefreq

(121.3519, 24.9546)

In [11]:
distance = 0.01
squarefreq = {}
#make out a dic that is the smallest boundScale = 1
for i in range(int(round(distance * 2,4)*10000)):
    for j in range(int(round(distance * 2,4)*10000)):
        squarefreq[tuple([round(downlat + i/10000 ,4), round(leftlong + j/10000,4)])] = 0

In [13]:
boundScale = 1
temptlist = df[['latitude','longitude']].values
for x in temptlist:
    # print(tuple(x), tuple(x) in squarefreq)
    if tuple(x) in squarefreq:
        #the way i think: use mod to divide into two group just like roundup
        if round((x[0] - downlat)*10000,0) % boundScale != 0:
            #less or equal than boundScale/2 = the smaller on 
            if round((x[0] - downlat)*10000,0)% boundScale <= boundScale/2:
                x[0] -= (round((x[0] - downlat)*10000,0) % boundScale)/10000
            else:
                x[0] += (boundScale - (round((x[0] - downlat)*10000,0) % boundScale))
            x[0] = round(x[0],4)
        if round((x[1] - leftlong)*10000,0) % boundScale != 0:
            if round((x[0] - leftlong)*10000,0) % boundScale != 0:
                x[1] -= (round((x[0] - leftlong)*10000,0) % boundScale)/10000
            else:
                x[1] += (boundScale - (round((x[0] - leftlong)*10000,0) % boundScale))
            x[1] = round(x[1],4)
        if tuple(x) in squarefreq:
            squarefreq[tuple(x)] += 1

In [14]:
squarefreq

{(24.9375, 121.3656): 0,
 (24.9375, 121.3657): 0,
 (24.9375, 121.3658): 0,
 (24.9375, 121.3659): 0,
 (24.9375, 121.366): 0,
 (24.9375, 121.3661): 0,
 (24.9375, 121.3662): 0,
 (24.9375, 121.3663): 0,
 (24.9375, 121.3664): 0,
 (24.9375, 121.3665): 0,
 (24.9375, 121.3666): 0,
 (24.9375, 121.3667): 0,
 (24.9375, 121.3668): 0,
 (24.9375, 121.3669): 0,
 (24.9375, 121.367): 0,
 (24.9375, 121.3671): 0,
 (24.9375, 121.3672): 0,
 (24.9375, 121.3673): 0,
 (24.9375, 121.3674): 0,
 (24.9375, 121.3675): 0,
 (24.9375, 121.3676): 0,
 (24.9375, 121.3677): 0,
 (24.9375, 121.3678): 0,
 (24.9375, 121.3679): 0,
 (24.9375, 121.368): 0,
 (24.9375, 121.3681): 0,
 (24.9375, 121.3682): 0,
 (24.9375, 121.3683): 0,
 (24.9375, 121.3684): 0,
 (24.9375, 121.3685): 0,
 (24.9375, 121.3686): 0,
 (24.9375, 121.3687): 0,
 (24.9375, 121.3688): 0,
 (24.9375, 121.3689): 0,
 (24.9375, 121.369): 0,
 (24.9375, 121.3691): 0,
 (24.9375, 121.3692): 0,
 (24.9375, 121.3693): 0,
 (24.9375, 121.3694): 0,
 (24.9375, 121.3695): 0,
 (24

In [15]:
       boundlist = []
#         # print(squarefreq)
        squarefreqMax = max(squarefreq.values())
#         # print(squarefreqMax)
        if squarefreqMax != 0:
            squarefreqMax = math.log10(max(squarefreq.values()))
        for k,values in squarefreq.items():
            key = list(k)
            org_values = values
            # print(org_values)
            if round((key[0] - downlat)*10000,0) % boundScale == 0:
                if round((key[1] - leftlong)*10000,0) % boundScale == 0:
                    lt = [round(key[0] + 0.00005*boundScale,5), round(key[1] - 0.00005*boundScale,5)]
                    rt = [round(key[0] + 0.00005*boundScale,5), round(key[1] + 0.00005*boundScale,5)]
                    rd = [round(key[0] - 0.00005*boundScale,5), round(key[1] + 0.00005*boundScale,5)]
                    ld = [round(key[0] - 0.00005*boundScale,5), round(key[1] - 0.00005*boundScale,5)]
                    precentage = 0
                    if values!=0:
                        values = math.log10(values)
                    if(values>=0)and(values<squarefreqMax/5):
                        precentage = 0
                    elif (values>=squarefreqMax/5)and(values<2*squarefreqMax/5):
                        precentage = 1
                    elif (values>=2*squarefreqMax/5)and(values<3*squarefreqMax/5):
                        precentage = 2
                    elif (values>=3*squarefreqMax/5)and(values<4*squarefreqMax/5):
                        precentage = 3
                    elif (squarefreqMax == 0):
                        precentage = 0
                    else:
                        precentage = 4
                    boundlist.append([lt,rt,rd,ld,precentage,org_values])
                    
        squarelen = int(math.sqrt(len(boundlist)))
        # print(squarelen,'\n\n////')
        spacelist = []
        #imply from bottomleft to topright
        # temptlist.append(boundlist[0][3])
        # temptlist.append(boundlist[0][2])
        # temptlist.append(boundlist[0][0])
        # temptlist.append(boundlist[0][1])

        # temptlist.append(boundlist[0][3])
        # temptlist.append(boundlist[10][1])
        # temptlist.append(boundlist[1][3])
        # temptlist.append([boundlist[10][1][0],boundlist[1][3][1]])

        #imply from topright to bottomleft
        # temptlist.append(boundlist[squarelen*squarelen - 1][1])
        # temptlist.append(boundlist[squarelen*squarelen - 1][3])
        # temptlist.append(boundlist[squarelen*squarelen - 1][2])
        # temptlist.append(boundlist[squarelen*squarelen - 1][0])

        # temptlist.append(boundlist[squarelen*squarelen - 2][0])
        # temptlist.append(boundlist[squarelen*(squarelen-1) - 1][2])
        # temptlist.append([boundlist[squarelen*(squarelen-1) - 1][3][0],boundlist[squarelen*squarelen - 2][0][1]])
        for x in range(squarelen):
            temptlist = []
            temptlist.append(boundlist[0][3])
            temptlist.append(boundlist[x * squarelen][0])
            temptlist.append([boundlist[x * squarelen][0][0],boundlist[x][2][1]])
            temptlist.append(boundlist[x][2])
            spacelist.append(temptlist)

        for x in range(squarelen - 1):
            temptlist = []
            temptlist.append(boundlist[squarelen*squarelen - 1][1])
            temptlist.append(boundlist[squarelen*squarelen - 1 - x][0])
            temptlist.append([boundlist[squarelen*(squarelen-x) - 1][3][0],boundlist[squarelen*squarelen - 1 - x][0][1]])
            temptlist.append(boundlist[squarelen*(squarelen-x) - 1][2])
            spacelist.append(temptlist)
            
        valuelist=[]
        # print(boundlist[0])
        for x in boundlist:
            # print(x[4],x[5])
            if x[4] != 0:
                # print(x[4])
                valuelist.append(x)
        valuelist = sorted(valuelist,key=lambda x: x[5],reverse=True)
#         sortNum = 1
        newlist = []
        # print(valuelist[0])
        # print(valuelist)
        for x in valuelist:
            tempt = x[0:6]
#             tempt.append(sortNum)
#             sortNum += 1
            newlist.append(tempt)

In [16]:
squarefreqMax

7361